In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as ticker
import seaborn as sns
import ast

In [37]:
artists_dataset = pd.read_csv('dataset/tabular/artists.csv')
tracks_dataset = pd.read_csv('dataset/tabular/tracks.csv')

# Artists Dataset

Dropping rows with NaN and duplicated lines from artists dataset

In [38]:
# Drop the rows with missing values
artists_dataset = artists_dataset.dropna()
# Drop the duplicated rows
artists_dataset = artists_dataset.drop_duplicates()

In [39]:
#Drop all the artists with same name and same genres
artists_dataset = artists_dataset.sort_values('popularity', ascending=False).drop_duplicates(['name', 'genres'])

For all rows with the same 'name' value, dropping the ones with empty 'genres' list and keeping the ones with non-empty 'genres' list

In [40]:
duplicated_names = artists_dataset[artists_dataset.duplicated('name', keep=False)]

# Assuming your DataFrame is named artists_dataset and the column with the genres list is named 'genres'
artists_dataset.loc[duplicated_names.index, 'genres'] = artists_dataset.loc[duplicated_names.index, 'genres'].apply(ast.literal_eval)  # Convert string representation of list to actual list
artists_dataset = artists_dataset[artists_dataset['genres'].apply(lambda x: len(x) != 0)]  # Keep only rows where 'genres' list is not empty

# Tracks Dataset

## Aggregated all rows with duplicated 'id' values into a single row, keeping the unique genres in a list. Then dropped the duplicated rows from the original dataset and added the aggregated rows.


In [41]:
merged_df = tracks_dataset.groupby('id')['genre'].agg(list)
df_merged = pd.merge(tracks_dataset, merged_df, on='id', how='left')

# Find the indices of the rows with the highest popularity within each group (ID)
indices_to_keep = df_merged.groupby('id')['popularity'].idxmax()
# Filter the dataframe to keep only the rows with the highest popularity within each group
tracks_dataset = df_merged.loc[indices_to_keep]
tracks_dataset.rename(columns={'genre_y': 'genre'}, inplace=True)
tracks_dataset = tracks_dataset.drop(columns=['genre_x'])
# Display the resulting dataframe
tracks_dataset

,id,name,disc_number,duration_ms,explicit,popularity,track_number,artists,album_type,album_name,...,features_duration_ms,time_signature,start_of_fade_out,tempo_confidence,time_signature_confidence,key_confidence,mode_confidence,n_beats,n_bars,genre
43417,0000vdREvCVMxbQTkS888c,Lolly,1,160725,True,35,1,Rill,single,Lolly,...,160726,4,154.11664,0.578,1.000,0.685,0.583,276.0,67.0,[german]
93608,000CC8EParg64OmTxVnZ0p,It's All Coming Back To Me Now (Glee Cast Vers...,1,322933,False,49,10,Glee Cast,album,Glee Love Songs,...,322933,4,313.21976,0.030,0.659,0.712,0.717,937.0,238.0,[club]
61659,000Iz0K615UepwSJ5z2RE5,Böxig Leise - Pig&Dan Remix,1,515360,False,0,5,Paul Kalkbrenner;Pig&Dan,album,X,...,515360,4,474.23273,0.832,0.996,0.052,0.322,1025.0,257.0,[minimal-techno]
423,000RDCYioLteXcutOjeweY,Teeje Week,1,190203,False,62,1,Jordan Sandhu;Bunty Bains,single,Teeje Week,...,190203,4,183.53633,0.103,1.000,0.339,0.473,501.0,124.0,[hip-hop]
31,000qpdoc97IMTBvF8gwcpy,Tief,1,331240,False,20,9,Paul Kalkbrenner,album,Zeit,...,331240,4,321.92145,0.622,1.000,0.011,0.160,699.0,175.0,[minimal-techno]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62633,7zxHiMmVLt4LGWpOMqOpUh,"Aethu Kari Raavilum - From ""Bangalore Days""",1,325156,False,64,1,Haricharan;Gopi Sundar,album,Bangalore Days,...,325156,4,315.87845,0.736,1.000,0.504,0.547,641.0,162.0,[pop-film]
106924,7zxpdh3EqMq2JCkOI0EqcG,"Two Worlds (From ""Tarzan"")",1,109573,False,31,3,Piano Genie,single,Disney Favourites,...,109574,4,105.35474,0.206,0.993,0.863,1.000,151.0,38.0,[disney]
58393,7zyYmIdjqqiX6kLryb7QBx,以後別做朋友,1,260573,False,64,2,Eric Chou,album,學著愛,...,260573,4,235.45615,0.270,0.822,0.395,0.533,539.0,135.0,[mandopop]
36699,7zybSU9tFO9HNlwmGF7stc,Sunset Drive,1,234300,False,60,5,Stereoclip,album,Echoes,...,234300,4,228.68753,0.958,0.895,0.493,0.233,485.0,121.0,[electronic]


## Handling songs with duplicated names

In [42]:
# Drop all the duplicates rows with same 'name' and 'artists' values, and keep only the row with the highest popularity
tracks_dataset = tracks_dataset.sort_values('popularity', ascending=False).drop_duplicates(['name', 'artists'])
# Display the resulting dataframe
tracks_dataset

,id,name,disc_number,duration_ms,explicit,popularity,track_number,artists,album_type,album_name,...,features_duration_ms,time_signature,start_of_fade_out,tempo_confidence,time_signature_confidence,key_confidence,mode_confidence,n_beats,n_bars,genre
13984,45OX2jjEw1l7lOFJfDP9fv,MONEY,1,168227,False,95,2,LISA,single,LALISA,...,168228,4,162.20880,0.789,0.989,0.043,0.253,381.0,95.0,"[k-pop, pop]"
5996,5XeFesFbtLpXzIVDNQP22n,I Wanna Be Yours,1,183956,False,94,12,Arctic Monkeys,album,AM,...,183956,4,174.09161,0.097,1.000,0.545,0.533,202.0,50.0,"[indie, garage, rock]"
83090,2bRKxuH1o7pTmb1y4GfdEc,Clean White Noise - Loopable with no fade,1,90228,False,94,1,White Noise Baby Sleep;White Noise for Babies,album,Best White Noise for Baby Sleep - Loopable wit...,...,90228,0,90.22821,0.000,0.000,0.000,0.116,0.0,0.0,[sleep]
62967,4uUG5RXrOk84mYEfFvj3cK,I'm Good (Blue),1,175238,True,94,1,David Guetta;Bebe Rexha,single,I'm Good (Blue),...,175238,4,163.43945,0.808,0.789,0.898,0.773,368.0,91.0,[pop]
62863,0WtM2NBVQNNJLh6scP13H8,Calm Down (with Selena Gomez),1,239317,False,93,1,Rema;Selena Gomez,single,Calm Down (with Selena Gomez),...,239318,4,231.09079,0.492,1.000,0.593,0.597,424.0,105.0,[pop]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95106,1nSYUQngGS5foiagiilbYU,Singles You Up,1,182693,False,0,3,Jordan Davis,compilation,Mientras hago aromaterapia,...,182693,4,177.22050,0.137,1.000,0.439,0.533,296.0,74.0,[country]
43751,1nU1tlW9ccptgRLfgq05I3,"Schlagzeile groß, Hirn zu klein",1,255093,False,0,26,Frei.Wild,album,Feinde deiner Feinde (Gold Edition),...,255093,4,247.60017,0.120,0.762,0.558,0.509,818.0,205.0,[german]
90963,72ZHywsIxzF1ZkNlMyhsqt,"Ding! Dong! The Witch Is Dead (From ""The Wizar...",1,179035,False,0,4,The Countdown Kids,album,Happy Halloween! (Spooky Favorites for Kids),...,179036,4,175.07846,0.826,0.959,0.715,0.532,352.0,87.0,[children]
16533,5Wt0gIt7RrXz5mJj5pslLC,Zenitsu Theme V2 (Thunder Clap and Flash!),1,137948,False,0,5,Samuel Kim,single,Demon Slayer: Epic Collection,...,137949,4,132.38857,0.545,1.000,0.791,0.703,263.0,67.0,[anime]


## Creating 3 new columns: 'release_year', 'release_month', 'release_day' from the 'release_date' column

In [43]:
# Extract year, month, and day from 'album_release_date'
# If 'album_release_date' is in YYYY-MM format, the day will be set as NaN

# First, ensure 'album_release_date' is a string to safely apply string operations
tracks_dataset['album_release_date'] = tracks_dataset['album_release_date'].astype(str)

# Split 'album_release_date' into year, month, and day
tracks_dataset['year'] = tracks_dataset['album_release_date'].apply(lambda x: x.split('-')[0])
tracks_dataset['month'] = tracks_dataset['album_release_date'].apply(lambda x: x.split('-')[1] if len(x.split('-')) > 1 else 'NaN')
tracks_dataset['day'] = tracks_dataset['album_release_date'].apply(lambda x: x.split('-')[2] if len(x.split('-')) > 2 else 'NaN')

# Display the first few rows to verify the new columns
tracks_dataset[['album_release_date', 'year', 'month', 'day']]

,album_release_date,year,month,day
13984,2021-09-10,2021,09,10
5996,2013-09-09,2013,09,09
83090,2017-01-08,2017,01,08
62967,2022-08-26,2022,08,26
62863,2022-08-25,2022,08,25
...,...,...,...,...
95106,2022-10-14,2022,10,14
43751,2013-04-19,2013,04,19
90963,2022-10-14,2022,10,14
16533,2021-05-21,2021,05,21


## Dropping columns track_number, disc_number, album_type, album_total_tracks 

In [44]:
# Drop the columns 'track_number', 'disc_number', 'album_type', 'album_total_tracks'
tracks_dataset = tracks_dataset.drop(columns=['track_number', 'disc_number', 'album_type', 'album_total_tracks', 'album_release_date_precision', 'album_release_date'])

In [45]:
# Export the cleaned datasets to CSV files
artists_dataset.to_csv('dataset/tabular/artists_cleaned.csv', index=False)
tracks_dataset.to_csv('dataset/tabular/tracks_cleaned.csv', index=False)